In [1]:
import numpy as np
import math
import cv2
import os
import joblib
from skimage.io import imread

In [2]:
from collections import Counter
width = 323
height = 241
data = joblib.load(f'car_surface_323x241px.pkl')

In [3]:
X = np.array(data['data'])
y = np.array(data['label'])

from sklearn.model_selection import train_test_split
 
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.3, 
    shuffle=True,
    random_state=76,
)

In [4]:
def fourd(x,y,w,h):
    dots = np.array([[x,y],[x+w,y],[x,y+h],[x+w,y+h]])
    return dots

In [5]:
def merge(x1,y1,w1,h1,x2,y2,w2,h2):
    dot1 = fourd(x1,y1,w1,h1)
    dot2 = fourd(x2,y2,w2,h2)
    dff = 77777
    for i in range(4):
        for j in range(4):
            df1 = dot1[i]-dot2[j]
            dff1 = math.hypot(df1[0],df1[1])
            if dff1 < dff:
                dff = dff1
    if dff > 15:
        return 0
    else:
        return 1

In [6]:
def inside(x1,y1,w1,h1,x2,y2,w2,h2):
    thre = 7
    for i in range(4):
        for j in range(4):
            if  (x1 < x2+thre) & (y1 < y2+thre) & (x1+w1+thre > x2+w2) & (y1+h1+thre > y2+h2):
                return 1
            if (x2 < x1+thre) & (y2 < y1+thre) & (x2+w2+thre > x1+w1) & (y2+h2+thre > y1+h1):
                return -1
            else:
                return 0

In [7]:
def white(xysize,imag):
    white = np.zeros(len(xysize))
    del_num = 0
    for i in range(len(xysize)):
        x,y,w,h = xysize[i]
        crop = imag[y:y+h, x:x+w]
        for j in range(crop.shape[1]):
            for k in range(crop.shape[0]):
                if crop[k][j] > 180:
                    white[i] = white[i]+1
        if white[i] < 50:
            imag[y:y+h, x:x+w] = np.zeros(crop.shape)
            del_num = del_num+1
    return white, del_num

In [8]:
def max_c(xysize,imag):
    max_c = -1
    tra = 0
    for i in range(len(xysize)):
        x,y,w,h = xysize[i]
        tra_c = w*h
        if tra_c > tra:
            max_c = i
            tra = tra_c
    if max_c == -1:
        return imag
    x,y,w,h = xysize[max_c]
    new_imag = np.zeros(imag.shape)
    new_imag[0:h, 0:w] = imag[y:y+h, x:x+w]
    return new_imag

In [9]:
from sklearn.base import BaseEstimator, TransformerMixin
from skimage.morphology import disk
from skimage.filters import median
from skimage import feature

class RGB2GrayTransformer(BaseEstimator, TransformerMixin):

    def __init__(self):
        pass
 
    def fit(self, X, y=None):
        return self
 
    def transform(self, X, y=None):
        def local_trans(X):
            src = X
            
            kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
            img_median = cv2.medianBlur(src, 5)
            blur = cv2.GaussianBlur(img_median,(5,5),0)
            open = cv2.morphologyEx(blur, cv2.MORPH_OPEN, kernel,iterations=1)
            close = cv2.morphologyEx(open, cv2.MORPH_CLOSE, kernel,iterations=1)

            ret,thre = cv2.threshold(close,close.mean()+15,255,cv2.THRESH_BINARY)

            contours,hierarchy = cv2.findContours(thre,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
            if len(contours) > 11:
                ret,thre = cv2.threshold(close,close.mean()+30,255,cv2.THRESH_BINARY)
                contours,hierarchy = cv2.findContours(thre,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
            two = np.zeros(src.shape)
            xysize = np.zeros((len(contours),4),dtype=int)
            for i in range(len(contours)):
                x,y,w,h = cv2.boundingRect(contours[i])
                xysize[i] = [x,y,w,h]
            
            white_num,del_num = white(xysize,thre)
            
            dele = np.zeros(del_num, dtype = int)
            count = 0
            for i in range(len(white_num)):
                if white_num[i] < 50:
                    dele[count] = i
                    count = count+1
            xysize = np.delete(xysize, dele, axis=0)
          
            
            i = 0
            ifdel = 0
            while i < len(xysize)-1:
                j = i+1
                ifdel = 0
                while j < len(xysize):
                    x1,y1,w1,h1 = xysize[i]
                    x2,y2,w2,h2 = xysize[j]
                    if merge(x1,y1,w1,h1,x2,y2,w2,h2):
                        cnt = np.concatenate((fourd(x2,y2,w2,h2),fourd(x1,y1,w1,h1)),axis=0)
                        xysize[i] = cv2.boundingRect(cnt)
                        xysize[i][2] = xysize[i][2]-1
                        xysize[i][3] = xysize[i][3]-1
                        xysize = np.delete(xysize, j, axis=0)
                        ifdel = 1
                        j = i+1
                    else:
                        j = j+1
                if ifdel == 0:
                    i = i+1

            i = 0
            ifdel = 0
            while i < len(xysize)-1:
                j = i+1
                ifdel = 0
                while j < len(xysize):
                    x1,y1,w1,h1 = xysize[i]
                    x2,y2,w2,h2 = xysize[j]
                    if inside(x1,y1,w1,h1,x2,y2,w2,h2) == 1:
                        xysize = np.delete(xysize, j, axis=0)
                        ifdel = 1
                        j = i+1
                    if inside(x1,y1,w1,h1,x2,y2,w2,h2) == -1:
                        xysize = np.delete(xysize, i, axis=0)
                        ifdel = 1
                        j = i+1
                    else:
                        j = j+1
                if ifdel == 0:
                    i = i+1

            thre = max_c(xysize, thre)
            thre = thre.astype('uint8')
            X = thre
            return X
    
        return np.array([local_trans(img) for img in X])

In [10]:
from skimage.transform import hough_line
from skimage.transform import hough_line_peaks
from skimage import transform

class feaTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, y=None):
        self.y = y
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        def local_fea(X):
            area = np.zeros(5)
            contours,hierarchy = cv2.findContours(X,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
            if len(contours) > 0:
                for i in range(len(contours)):
                    cnt = contours[i]
                    area[0] = area[0] + cv2.contourArea(cnt)
                    area[1] = area[1] + cv2.arcLength(cnt,True)
                area[2] = area[1]/area[0]
                hspace, angles, dists = hough_line(X)
                hspace, angles, dists = hough_line_peaks(hspace, angles, dists)
                area[3] = len(angles)
                radius = 8
                hspaces = transform.hough_circle(X, radius)
                accum, cx, cy, rad = transform.hough_circle_peaks(hspaces, [radius,])
                area[4] = len(rad)
            return area
        return np.array([local_fea(img) for img in X])

In [11]:
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import StandardScaler, Normalizer
import skimage

grayify = RGB2GrayTransformer()
scalify = StandardScaler()
feaify = feaTransformer()

X_train_gray = grayify.fit_transform(X_train)
X_train_fea = feaify.fit_transform(X_train_gray)
X_train_prepared = scalify.fit_transform(X_train_fea)

from sklearn.ensemble import RandomForestClassifier
clf = joblib.load('clfte.joblib')

In [12]:
impath = ()

In [13]:
import tkinter as tk
from tkinter.filedialog import (askopenfilename, 
                                    askopenfilenames, 
                                    askdirectory, 
                                    asksaveasfilename)
from PIL import Image, ImageTk
    
def imselect():
    for widgets in frame.winfo_children():
        widgets.destroy()
    impath = askopenfilenames()
    if len(impath) > 0:
        for i in range(len(impath)):
            
            X_test = np.array([cv2.imread(impath[i], 0)])
            X_test_gray = grayify.transform(X_test)
            X_test_fea = feaify.transform(X_test_gray)
            X_test_prepared = scalify.transform(X_test_fea)
            y_pred = clf.predict(X_test_prepared)
            load = Image.open(impath[i])
            img_label = tk.Label(frame)
            img_label.image = ImageTk.PhotoImage(load)
            img_label['image'] = img_label.image
            img_label.pack()
            label = tk.Label(frame, text=y_pred[0])
            label.pack()

                

window = tk.Tk()
window.title('Classifier')
window.geometry('600x900')

select = tk.Button(window, text="选择", command=imselect)
select.pack()

frame = tk.Frame()
frame.pack()

window.mainloop()